# Business Understanding

# Data Engineering

## Import Libraries

In [32]:
# import libraries required to load, transform, analyze and plot data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(context='paper', style='darkgrid', 
        rc={'figure.facecolor':'white'}, font_scale=1.2)

import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from spacy.tokenizer import _get_regex_pattern

In [2]:
# remove scientific notation and restrictions on df rows/columns display
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('max_rows', 200)
pd.set_option('display.max_colwidth', 150)

## Dataframe Basics

### Load File

In [3]:
# load primary source file to df, renaming columns, dropping non-ASCII
col_names = ['tweet_text', 'directed_at', 'emotion_label']
tweets = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding= 'unicode_escape', names=col_names, header=0)
tweets.head()

,tweet_text,directed_at,emotion_label
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


### Data Values

In [21]:
# review data types and null counts
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_text        9092 non-null   object
 1   directed_at       9093 non-null   object
 2   emotion_label     9093 non-null   object
 3   brand             9093 non-null   object
 4   tweet_text_clean  9092 non-null   object
 5   tokens            9093 non-null   object
 6   tokens_sp         9093 non-null   object
dtypes: object(7)
memory usage: 497.4+ KB


In [27]:
# drop nan tweets from dataframe
tweets.dropna(subset = ['tweet_text'], inplace=True)
tweets.shape

(9092, 7)

In [5]:
# check value counts by column
print(tweets['directed_at'].value_counts(normalize=True, dropna=False))

NaN                               0.638
iPad                              0.104
Apple                             0.073
iPad or iPhone App                0.052
Google                            0.047
iPhone                            0.033
Other Google product or service   0.032
Android App                       0.009
Android                           0.009
Other Apple product or service    0.004
Name: directed_at, dtype: float64


In [6]:
# create brand feature
tweets['directed_at'].fillna('None', inplace=True)
brand_map = {'iPad': 'Apple', 'Apple': 'Apple', 'iPad or iPhone App': 'Apple', 
             'Google': 'Google', 'iPhone': 'Apple', 
             'Other Google product or service': 'Google',
            'Android App': 'Google', 'Android': 'Google',
             'Other Apple product or service': 'Apple',
             'None': 'None'
            }
tweets['brand'] = tweets.directed_at.map(brand_map, na_action='ignore')

In [7]:
# clean emotion labels
tweets['emotion_label'].replace({'No emotion toward brand or product': 'Neutral',
                                 'Positive emotion': 'Positive', 
                                 'Negative emotion': 'Negative', 
                                 'I can\'t tell': 'Unknown'}, inplace=True)

# check value counts by column
print(tweets['emotion_label'].value_counts(normalize=True, dropna=False))

Neutral    0.593
Positive   0.328
Negative   0.063
Unknown    0.017
Name: emotion_label, dtype: float64


In [8]:
# check value counts by column
tweets.groupby(by=['brand', 'emotion_label'])['tweet_text'].count()

brand   emotion_label
Apple   Negative          388
        Neutral            65
        Positive         1949
        Unknown             7
Google  Negative          131
        Neutral            26
        Positive          723
        Unknown             2
None    Negative           51
        Neutral          5297
        Positive          306
        Unknown           147
Name: tweet_text, dtype: int64

## Tweet Text Clean

In [9]:
# Get all the stop words and punctuation in the English language
stops = stopwords.words('english')
punctuation = list(string.punctuation)
punctuation.remove('#') # keep # for hashtags

# sample stopwods and punctuations
print(f'Stopword Count: {len(stops)} Sample 5: {stops[0:5]}')
print(f'Punctuation Count: {len(punctuation)} Sample 5: {punctuation[0:5]}')

Stopword Count: 179 Sample 5: ['i', 'me', 'my', 'myself', 'we']
Punctuation Count: 31 Sample 5: ['!', '"', '$', '%', '&']


In [10]:
def remove_punctuation(x):
    """
    Helper function to remove punctuation from a string x: any string
    """
    try:
        x = re.sub('@[A-Za-z0-9]+', '', x) # remove @mention users
        x = re.sub(r'http\S+', '', x) # remove URL references
        x = re.sub(r'\b[0-9]+\b', '', x) # remove stand-alone numbers  
        x = ''.join(ch for ch in x if ch not in punctuation) # remove punc
    except:
        pass
    return x

In [11]:
# function to clean text
def  clean_text(df, text_field, new_text_field):
    df[new_text_field] = df[text_field].str.lower()
    df[new_text_field] = df[new_text_field].apply(remove_punctuation) 
    return df

In [12]:
tweets_clean = clean_text(tweets, 'tweet_text', 'tweet_text_clean')
tweets_clean.head(20)

,tweet_text,directed_at,emotion_label,brand,tweet_text_clean
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative,Apple,i have a 3g iphone after hrs tweeting at #riseaustin it was dead i need to upgrade plugin stations at #sxsw
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive,Apple,know about awesome ipadiphone app that youll likely appreciate for its design also theyre giving free ts at #sxsw
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive,Apple,can not wait for #ipad also they should sale them down at #sxsw
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative,Apple,i hope this years festival isnt as crashy as this years iphone app #sxsw
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive,Google,great stuff on fri #sxsw marissa mayer google tim oreilly tech booksconferences amp matt mullenweg wordpress
5,@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd,None,Neutral,None,new ipad apps for #speechtherapy and communication are showcased at the #sxsw conference #iear #edchat #asd
6,NaN,None,Neutral,None,NaN
7,"#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan",Android,Positive,Google,#sxsw is just starting #ctia is around the corner and #googleio is only a hop skip and a jump from there good time to be an #android fan
8,Beautifully smart and simple idea RT @madebymany @thenextweb wrote about our #hollergram iPad app for #sxsw! http://bit.ly/ieaVOB,iPad or iPhone App,Positive,Apple,beautifully smart and simple idea rt wrote about our #hollergram ipad app for #sxsw
9,Counting down the days to #sxsw plus strong Canadian dollar means stock up on Apple gear,Apple,Positive,Apple,counting down the days to #sxsw plus strong canadian dollar means stock up on apple gear


In [13]:
tk = TweetTokenizer()
tweets['tokens'] = tweets.tweet_text_clean.apply(str).apply(tk.tokenize)
tweets['tokens'] = tweets.tokens.apply(lambda x: [w for w in x if w not in stops])
tweets.head()

,tweet_text,directed_at,emotion_label,brand,tweet_text_clean,tokens
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative,Apple,i have a 3g iphone after hrs tweeting at #riseaustin it was dead i need to upgrade plugin stations at #sxsw,"[3g, iphone, hrs, tweeting, #riseaustin, dead, need, upgrade, plugin, stations, #sxsw]"
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive,Apple,know about awesome ipadiphone app that youll likely appreciate for its design also theyre giving free ts at #sxsw,"[know, awesome, ipadiphone, app, youll, likely, appreciate, design, also, theyre, giving, free, ts, #sxsw]"
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive,Apple,can not wait for #ipad also they should sale them down at #sxsw,"[wait, #ipad, also, sale, #sxsw]"
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative,Apple,i hope this years festival isnt as crashy as this years iphone app #sxsw,"[hope, years, festival, isnt, crashy, years, iphone, app, #sxsw]"
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive,Google,great stuff on fri #sxsw marissa mayer google tim oreilly tech booksconferences amp matt mullenweg wordpress,"[great, stuff, fri, #sxsw, marissa, mayer, google, tim, oreilly, tech, booksconferences, amp, matt, mullenweg, wordpress]"


In [30]:
# function
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

tweets['tokens_lem'] = tweets.tokens.apply(lemmatize_text)
tweets.head()

,tweet_text,directed_at,emotion_label,brand,tweet_text_clean,tokens,tokens_sp,tokens_lem
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative,Apple,i have a 3g iphone after hrs tweeting at #riseaustin it was dead i need to upgrade plugin stations at #sxsw,"[3g, iphone, hrs, tweeting, #riseaustin, dead, need, upgrade, plugin, stations, #sxsw]","( , I, have, a, 3, G, iPhone, After, , hrs, tweeting, at, #, RISEAustin, it, was, dead, , I, need, to, upgrade, Plugin, stations, at, #, SXSW)","[3g, iphone, hr, tweeting, #riseaustin, dead, need, upgrade, plugin, station, #sxsw]"
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive,Apple,know about awesome ipadiphone app that youll likely appreciate for its design also theyre giving free ts at #sxsw,"[know, awesome, ipadiphone, app, youll, likely, appreciate, design, also, theyre, giving, free, ts, #sxsw]","( , Know, about, , Awesome, iPadiPhone, app, that, you, ll, likely, appreciate, for, its, design, Also, they, re, giving, free, Ts, at, #, SXSW)","[know, awesome, ipadiphone, app, youll, likely, appreciate, design, also, theyre, giving, free, t, #sxsw]"
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive,Apple,can not wait for #ipad also they should sale them down at #sxsw,"[wait, #ipad, also, sale, #sxsw]","( , Can, not, wait, for, #, iPad, , also, They, should, sale, them, down, at, #, SXSW)","[wait, #ipad, also, sale, #sxsw]"
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative,Apple,i hope this years festival isnt as crashy as this years iphone app #sxsw,"[hope, years, festival, isnt, crashy, years, iphone, app, #sxsw]","( , I, hope, this, years, festival, is, nt, as, crashy, as, this, years, iPhone, app, #, sxsw)","[hope, year, festival, isnt, crashy, year, iphone, app, #sxsw]"
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive,Google,great stuff on fri #sxsw marissa mayer google tim oreilly tech booksconferences amp matt mullenweg wordpress,"[great, stuff, fri, #sxsw, marissa, mayer, google, tim, oreilly, tech, booksconferences, amp, matt, mullenweg, wordpress]","( , great, stuff, on, Fri, #, SXSW, Marissa, Mayer, Google, Tim, OReilly, tech, booksconferences, amp, Matt, Mullenweg, Wordpress)","[great, stuff, fri, #sxsw, marissa, mayer, google, tim, oreilly, tech, booksconferences, amp, matt, mullenweg, wordpress]"


In [37]:
nlp = spacy.load('en_core_web_sm')
# get default pattern for tokens that don't get split
re_token_match = _get_regex_pattern(nlp.Defaults.token_match)
# add your patterns (here: hashtags and in-word hyphens)
re_token_match = f"({re_token_match}|#\w+|\w+-\w+)"

# overwrite token_match function of the tokenizer
nlp.tokenizer.token_match = re.compile(re_token_match).match

In [50]:

stops_sp = nlp.Defaults.stop_words
print(f'spaCy Stopword Count: {len(stops_sp)}')

def remove_stops(doc):
    return [token.lemma_ for token in doc if not token.is_stop if not token.is_punct]

tweets['tokens_sp'] = [nlp(row) for row in tweets.tweet_text_clean.apply(str)]
tweets['tokens_sp2'] = [remove_stops(doc) for doc in tweets.tokens_sp ]

if not token.is_punct

tweets.head()
#     practice = "practice practiced practicing" 
# ...: nlp_practice = nlp(practice) 
# ...: [word.lemma_ for word in nlp_practice]

SyntaxError: invalid syntax (<ipython-input-50-c15a54e1377d>, line 10)

In [44]:
test = tweets.tokens_sp[0]
for token in test:
    token.token, token.lemma_ 

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'token'

In [ ]:
lemma_word1 = [] 
for token in doc:
    lemma_word1.append(token.lemma_)
lemma_word1

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)

from spacy.lang.en.stop_words import STOP_WORDS

# Create list of word tokens after removing stopwords
filtered_sentence =[] 

for word in token_list:
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        filtered_sentence.append(word) 
print(token_list)
print(filtered_sentence) 

In [95]:
# 
word_dict = {}

# Loop through all the tags
for i, row in tweets['tokens'].iteritems():
    for word in row:
        if word not in word_dict:
            word_dict[word] = 1
        else:
            word_dict[word] +=1

word_counts = sorted(word_dict.items(), key=lambda kv: kv[1], reverse=True)
print(f'Total words: {len(word_counts)}')
word_counts[:25]

Total words: 10925


[('#sxsw', 9057),
 ('link', 4284),
 ('rt', 2957),
 ('ipad', 2333),
 ('google', 2026),
 ('apple', 1744),
 ('store', 1465),
 ('iphone', 1251),
 ('new', 1088),
 ('austin', 842),
 ('app', 789),
 ('amp', 724),
 ('\x89', 691),
 ('launch', 643),
 ('social', 623),
 ('popup', 597),
 ('today', 563),
 ('circles', 528),
 ('sxsw', 490),
 ('network', 462),
 ('via', 435),
 ('android', 434),
 ('#apple', 410),
 ('line', 399),
 ('get', 393)]

In [ ]:
total_vocabulary = set(word for headline in data for word in headline)

len(total_vocabulary)
print('There are {} unique tokens in the dataset.'.format(len(total_vocabulary)))

glove = {}
with open('glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

# Data Analysis

# Predictive Models

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [ ]:
y = pd.get_dummies(target).values

In [ ]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(df['combined_text']))
list_tokenized_headlines = tokenizer.texts_to_sequences(df['combined_text'])
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [ ]:
model = Sequential()
embedding_size = 128
model.add(Embedding(20000, embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(41, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_t, y, epochs=3, batch_size=32, validation_split=0.1)